In [1]:
from google.colab import drive

folder = 'project_VeP'#@param{type:"string"}

drive.mount('/content/gdrive', force_remount=True)
print('Connecting to Drive....')

%cd /content/gdrive/MyDrive/$folder/
print('Done!')

Mounted at /content/gdrive
Connecting to Drive....
/content/gdrive/MyDrive/project_VeP
Done!


In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
import torch
print("Repo Clone Completed!")

In [ ]:
epoch = 300 #@param {type:"slider", min:100, max:1000, step:10}
training_option = 'Train new CNN' #@param ["Train new CNN", "Resume an interrupted Training"]
weight = 'yolov5m.pt' #@param["yolov5s.pt","yolov5m.pt","yolov5l.pt","yolov5x.pt"]
batch = 64 #@param {type:"slider", min:1, max:64, step:1}

if training_option == 'Train new CNN':
  !python /content/gdrive/MyDrive/$folder/yolov5/train.py --img 512 --batch $batch --epochs $epoch --data /content/gdrive/MyDrive/$folder/training_dataset/data.yaml --weights $weight --cache --project /content/gdrive/MyDrive/$folder/training_results/
else:
  !python /content/gdrive/MyDrive/$folder/yolov5/train.py --resume /content/gdrive/MyDrive/$folder/training_results/export/weights/last.pt

#Metriche
Il codice seguente calcola le metriche su delle immagini di test. Vengono poi confrontati i grafici della curva per la F1, sia nel caso del training che sulle immagini di test.

In [ ]:
!python /content/gdrive/MyDrive/$folder/yolov5/val.py --data /content/gdrive/MyDrive/$folder/test_dataset/data.yaml --weights /content/gdrive/MyDrive/$folder/training_results/exp2/weights/best.pt --project /content/gdrive/MyDrive/$folder/test_results/ --img 512 --iou 0.65 --half

In [ ]:
#Printing result to confront
from PIL import Image

precision_training = '/content/gdrive/MyDrive/'+folder+'/training_results/exp5/F1_curve.png'
precision_test = '/content/gdrive/MyDrive/'+folder+'/test_results/exp2/F1_curve.png'

im1 = Image.open(precision_training)
im2 = Image.open(precision_test)

print("F1 Training")
display(im1)
print("F1 Test")
display(im2)

#Detection
Effettuiamo la detection con YoloV5, definiamo la funzione che ci permette di fare il Merge delle bounding box estratte, e infine calcoliamo le inferenze con eventuali merge.

In [ ]:
!python /content/gdrive/MyDrive/$folder/yolov5/detect.py --source /content/gdrive/MyDrive/$folder/detection/exp --weights /content/gdrive/MyDrive/$folder/training_results/exp/weights/best.pt --project /content/gdrive/MyDrive/$folder/detection_results/ --imgsz 512 --save-txt --save-conf --exist-ok --nosave

In [3]:
import glob
import os
import shutil
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

#Bounding Box Merge Algorithm
def merge_bounding_boxes(bounding_boxes,image_width,image_height):
  [x,y] = [image_height,image_width]
  [w,h] = [0,0]
  for i in range(0,bounding_boxes.shape[0]):
    if(bounding_boxes.ndim == 1):
      center_x_1 = int(bounding_boxes[0]*image_width)
      center_y_1 = int(bounding_boxes[1]*image_height) 
      width_1 = int(bounding_boxes[2]*image_width) 
      height_1 = int(bounding_boxes[3]*image_height)
      x = int(center_x_1 - (width_1/2))
      y = int(center_y_1 - (height_1/2))
      w = int(center_x_1 + (width_1/2))
      h = int(center_y_1 + (height_1/2))
    else:
      center_x_1 = int(bounding_boxes[i,0]*image_width)
      center_y_1 = int(bounding_boxes[i,1]*image_height) 
      width_1 = int(bounding_boxes[i,2]*image_width) 
      height_1 = int(bounding_boxes[i,3]*image_height)
      vert_x = int(center_x_1 - (width_1/2))
      vert_y = int(center_y_1 - (height_1/2))
      vert_w = int(center_x_1 + (width_1/2))
      vert_h = int(center_y_1 + (height_1/2))
      if(vert_x < x):
        x = vert_x
      if(vert_y < y):
        y = vert_y
      if(vert_w > w):
        w = vert_w
      if(vert_h > h):
        h = vert_h
    return np.array([x,y,w,h])

In [ ]:
path = '/content/gdrive/MyDrive/'+folder+'/detection_results/exp/labels/'
img_path = '/content/gdrive/MyDrive/'+folder+'/detection/exp/'
save_path = '/content/gdrive/MyDrive/'+folder+'/detection_results/exp/'

#shutil.copytree(img_path, save_path)

for filename in glob.glob(os.path.join(path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename), 'r') as f:
    bounding_boxes = None
    confidences = None
    image_name = os.path.basename(filename).replace('txt', 'jpg')
    for line in f:
      cl, label_x, label_y, label_w, label_h, conf = line.split(' ')
      b = float(conf)
      a = np.array([float(label_x),float(label_y),float(label_w),float(label_h)])
      bounding_boxes = (np.vstack((bounding_boxes, a)) if (bounding_boxes is not None) else a)
      confidences = (np.vstack((confidences, b)) if (confidences is not None) else b)
    #Image Elaboration
    image = cv.imread(img_path+image_name)
    [image_height, image_width, levels] = image.shape
    conf_max = np.amax(confidences)
    [x,y,w,h] = merge_bounding_boxes(bounding_boxes, image_width, image_height)
    #Create Rect
    cv.rectangle(image, (x,y), (w,h), (0,0,255), 4)
    cv.putText(image, 'crosswalk ' + "%.2f" % conf_max, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    #Saving Image
    print('Saving ' + image_name)
    cv.imwrite(save_path+image_name, image)

#Intersection Over Union (IoU)
A questo punto calcoliamo la metrica IoU con le nostre inferenze. Abbiamo definito noi una nostra funzione per effettuare il calcolo

In [31]:
#INTERESCTION OVER UNION (IOU)
def compute_iou(box1, box2):
  #DATA: box1 = [L1, T1, R1, B1], box2 = [L2, T2, R2, B2]
  L_inter = max(box1[0], box2[0])
  T_inter = max(box1[1], box2[1])
  R_inter = min(box1[2], box2[2])
  B_inter = min(box1[3], box2[3])
  if(R_inter < L_inter) or (B_inter < T_inter):
    return 0
  A_inter = (R_inter - L_inter) * (B_inter - T_inter)
  A1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
  A2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
  A_union = A1 + A2 - A_inter
  iou = A_inter/A_union
  return iou

#READING DATA AND CALCULATING IOU
dataset_path = '/content/gdrive/MyDrive/'+folder+'/intersection_over_union/dataset/'
detection_path = '/content/gdrive/MyDrive/'+folder+'/detection_results/exp/labels/'
images_path = '/content/gdrive/MyDrive/'+folder+'/detection/exp/'

iou_array = None

for filename_1 in glob.glob(os.path.join(dataset_path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename_1), 'r') as f:
    name_1 = os.path.basename(filename_1)
    for filename_2 in glob.glob(os.path.join(detection_path, '*.txt')):
      name_2 = os.path.basename(filename_2)
      with open(os.path.join(os.getcwd(),filename_2), 'r') as g:
        if(name_1 == name_2):
          image_name = os.path.basename(filename_2).replace('txt', 'jpg')
          bounding_boxes_1 = None
          bounding_boxes_2 = None
          for line in f:
            cl_1, label_x_1, label_y_1, label_w_1, label_h_1 = line.split(' ')
            a_1 = np.array([float(label_x_1),float(label_y_1),float(label_w_1),float(label_h_1)])
            bounding_boxes_1 = (np.vstack((bounding_boxes_1, a_1)) if (bounding_boxes_1 is not None) else a_1)
          for line in g:
            cl_2, label_x_2, label_y_2, label_w_2, label_h_2, conf_2 = line.split(' ')
            a_2 = np.array([float(label_x_2),float(label_y_2),float(label_w_2),float(label_h_2)])
            bounding_boxes_2 = (np.vstack((bounding_boxes_2, a_2)) if (bounding_boxes_2 is not None) else a_2)
          image = cv.imread(images_path+image_name)
          [image_height, image_width, levels] = image.shape
          merge_box_1 = merge_bounding_boxes(bounding_boxes_1, image_width, image_height)
          merge_box_2 = merge_bounding_boxes(bounding_boxes_2, image_width, image_height)
          iou = compute_iou(merge_box_1, merge_box_2)
          iou_array = (np.vstack((iou_array, iou)) if (iou_array is not None) else iou)
        else:
          continue
print('IoU: ' + str(iou_array.mean())[:4])

IoU: 0.77


#Video Detection
Anche qui effettuiamo la detection con YoloV5, e andiamo a sostituire i frame con le detection di YoloV5 con i frame contenenti le nostre bounding box dopo il merge.

In [ ]:
#VIDEO DETECTION
video = 'video1.mp4' #@param {type:"string"}
!python /content/gdrive/MyDrive/$folder/yolov5/detect.py --source /content/gdrive/MyDrive/$folder/video/$video --weights /content/gdrive/MyDrive/$folder/training_results/exp/weights/best.pt --project /content/gdrive/MyDrive/$folder/video_results/ --imgsz 512 --save-txt --save-conf --exist-ok #--nosave

In [25]:
#CREATING INFERENCE ON VIDEO
import glob
import os
import shutil
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

path = '/content/gdrive/MyDrive/'+folder+'/video_results/exp/labels/'
video_path = '/content/gdrive/MyDrive/'+folder+'/video/'
save_path = '/content/gdrive/MyDrive/'+folder+'/video_results/exp/'
frames_path = '/content/gdrive/MyDrive/'+folder+'/video_results/exp/frames/'
video_name = video.replace('.mp4', '')

print('Reading Video: '+video_path+video)
v = cv.VideoCapture(video_path+video)
frame_count = int(v.get(cv.CAP_PROP_FRAME_COUNT))
fps = v.get(cv.CAP_PROP_FPS)
video_w = int(v.get(cv.CAP_PROP_FRAME_WIDTH))
video_h = int(v.get(cv.CAP_PROP_FRAME_HEIGHT))
print('Video Info' + '  FPS: '+str(fps)+' Width: '+str(video_w)+' Height: '+str(video_h))

frames = None
for filename in glob.glob(os.path.join(path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename), 'r') as f:
    bounding_boxes = None
    confidences = None
    #Saves the Frame number in an array
    frame_num = int(os.path.basename(filename).replace('.txt', '').replace(video_name +'_',''))
    frames = (np.vstack((frames, frame_num)) if (frames is not None) else frame_num)
    for line in f:
      cl, label_x, label_y, label_w, label_h, conf = line.split(' ')
      b = float(conf)
      a = np.array([float(label_x),float(label_y),float(label_w),float(label_h)])
      bounding_boxes = (np.vstack((bounding_boxes, a)) if (bounding_boxes is not None) else a)
      confidences = (np.vstack((confidences, b)) if (confidences is not None) else b)
    conf_max = np.amax(confidences)
    #Get Frames from Video
    v.set(cv.CAP_PROP_POS_FRAMES, frame_num)
    res, image = v.read()
    image_height = video_h
    image_width = video_w
    [x,y,w,h] = merge_bounding_boxes(bounding_boxes, image_width, image_height)
    cv.rectangle(image, (x,y), (w,h), (0,0,255), 4)
    cv.putText(image, 'crosswalk ' + "%.2f" % conf_max, (x, y+20), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    if image is None:
      continue
    cv.imwrite(frames_path+str(frame_num)+'.jpg', image)
v.release()

#SAVING NEW VIDEO INFERENCES
cap = cv.VideoCapture(video_path+video)
vid_writer = cv.VideoWriter(save_path+video, cv.VideoWriter_fourcc(*'mp4v'), fps, (video_w, video_h))
frame_num = -1;
while (cap.isOpened()):
  frame_num +=1
  ret, frame = cap.read()
  if ret == True:
    if(frame_num in frames):
      frame = cv.imread('/content/gdrive/MyDrive/'+folder+'/video_results/exp/frames/'+str(frame_num)+'.jpg')
    vid_writer.write(frame)
  else:
    break
cap.release()
vid_writer.release()
print('Result Saved on '+save_path+video)

NameError: ignored